In [ ]:
%run preprocessing.py
print ("shape of usersXprod :", usersXprod.shape)
print (usersXprod.columns)


> 對於購買力很差的顧客做篩選  
針對一些超愛reordered跟超愛亂買的消費者做刪除 (high, normal, low)

In [ ]:
# (train, test): (131209, 75000)

t_afew = 20
t_low = 0.1

afewXlow = usersXprod[(usersXprod.eval_set == "train")&(usersXprod.user_total_product <= t_afew)&(usersXprod.user_reorder_ratio < t_low)]
afew = usersXprod[(usersXprod.eval_set == "train")&(usersXprod.user_total_product <= t_afew)]
low = usersXprod[(usersXprod.eval_set == "train")&(usersXprod.user_reorder_ratio < t_low)]

print ("user of afewXlow :", len(set(afewXlow.user_id)))
print ("user of afew :", len(set(afew.user_id)))
print ("user of low :", len(set(low.user_id)))
print ("usersXprod of afewXlow :", afewXlow.shape[0]/usersXprod[(usersXprod.eval_set == "train")].shape[0])
print ("usersXprod of afew :", afew.shape[0]/usersXprod[(usersXprod.eval_set == "train")].shape[0])
print ("usersXprod of low :", low.shape[0]/usersXprod[(usersXprod.eval_set == "train")].shape[0])

del afewXlow
del afew
del low

In [ ]:
con_train = (usersXprod.eval_set == "train")&(usersXprod.user_reorder_ratio >= t_low)
con_test = (usersXprod.eval_set == "test")

train = usersXprod[con_train]
train = train.drop(["eval_set", "user_id", "product_id", "order_id"], axis=1)

test0 = usersXprod[con_test]
test = test0.drop(["eval_set", "user_id", "product_id", "order_id", "reordered"], axis=1)

#del usersXprod
print("size of training set :", train.shape[0])
print("size of testing set :", test.shape[0])


In [ ]:
X = train.drop(["reordered"], axis = 1)
y = train[["reordered"]]

class_weight_0 = (y.shape[0] - np.sum(y.reordered)) / y.shape[0]
class_weight_1 = np.sum(y.reordered) / y.shape[0]

print("Size of features of dataset :", X.shape[1])
print("Class weight of 0 :", class_weight_0)
print("Class weight of 1 :", class_weight_1)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier


In [ ]:
sc = StandardScaler(with_mean = True, with_std = True)
X_std = sc.fit_transform(X)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
test_std = sc.fit_transform(test)
threshold = 0.1


In [ ]:
def validation(model):
    model.fit(X_train, y_train)
    print(str(model), "cross_val down")
    y_pred = model.predict(X_val).reshape(y_val.shape[0], 1)
    print('Misclassified samples: %d' % (y_val != y_pred).sum())
    print('Accuracy : %.2f' % ((y_val == y_pred).sum() / y_val.shape[0]))
    print('Accuracy (sklearn): %.2f' % accuracy_score(y_val, y_pred))
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y_val, y_pred, labels = [0,1], sample_weight = None))
    print("=== F1-score ===")
    print(f1_score(y_val, y_pred, labels=[0,1]))

# 整理過的testing dataset 去預測
# 拿出以前的商品顧客組合去預測是否reordered    
# 建立一個門檻值去篩選是否購買???
# 選出指定order_id 購買的商品並匯出
def cross_val_and_testing(df, df_test, model, threshold):
    model.fit(X, y)
    print (cross_val_score(model, np.array(X), np.array(y).reshape(y.shape[0], ), cv=5))
    print ("Fitting Over")
    pred = pd.DataFrame(model.predict_proba(df_test), columns = ["prob_0", "prob_1"])
    df["prediction"] = pred["prob_1"]
    df = df[["order_id", "product_id", "prediction"]]

    con = (df.prediction > threshold)
    test0_0 = df[~con]
    test0_1 = df[con]
    print("Percent of test0 is 0 :", test0_0.shape[0]/df.shape[0])
    print("Percent of test0 is 1 :", test0_1.shape[0]/df.shape[0])

    f = lambda x: np.array_split(list(set(x)), 1)[0]
    order_prod = test0_1.groupby('order_id')["product_id"].apply(f).reset_index()
    order_id = df.groupby("order_id")["product_id"].apply(f).reset_index()
    submit = pd.merge(order_id[["order_id"]], order_prod, how='left', on=['order_id'])
    return submit


In [ ]:
lg = LogisticRegression(penalty = 'l2',
                        dual = False,
                        tol = 0.0001,
                        C = 0.1,
                        fit_intercept = True,
                        intercept_scaling = 1,
                        class_weight = {1: class_weight_1, 0: class_weight_0},
                        random_state = 100,
                        solver = 'liblinear',
                        max_iter = 10,
                        multi_class = 'ovr',
                        verbose = 0,
                        warm_start = False,
                        n_jobs = 1)

tree = DecisionTreeClassifier(criterion = 'entropy', 
                              max_depth = None, 
                              random_state = 0,
                              class_weight = {1: class_weight_1, 0: class_weight_0})

tree_3 = DecisionTreeClassifier(criterion = 'entropy', 
                                max_depth = 3, 
                                random_state = 0,
                                class_weight = {1: class_weight_1, 0: class_weight_0})

rfc = RandomForestClassifier(n_estimators = 10,
                             criterion = 'entropy',
                             max_depth = None,
                             min_samples_split = 2,
                             min_samples_leaf = 1,
                             min_weight_fraction_leaf = 0.0,
                             max_features = 'auto',
                             max_leaf_nodes = None,
                             min_impurity_split = 1e-07,
                             bootstrap = True,
                             oob_score = False,
                             n_jobs = 1,
                             random_state = 100,
                             verbose = 0,
                             warm_start = False,
                             class_weight = {1: class_weight_1, 0: class_weight_0})

bc = BaggingClassifier(base_estimator = tree,  
                       n_estimators = 10,  
                       max_samples = 1.0,  
                       max_features = 1.0,  
                       bootstrap = True,  
                       bootstrap_features = False,  
                       oob_score = False,  
                       warm_start = False,  
                       n_jobs = 1,  
                       random_state = None,  
                       verbose = 0)

abc = AdaBoostClassifier(base_estimator = tree_3,
                         n_estimators = 50,
                         learning_rate = 1.0,
                         algorithm = 'SAMME.R',
                         random_state = 100)

vc = VotingClassifier(estimators = [("rfc", rfc), ("bc", bc), ("abc", abc)],
                      voting = "soft",
                      weights = [2, 2, 1])


In [ ]:
validation(tree)
submit = cross_val_and_testing(df=test0, df_test=test, model=tree, threshold=threshold)
submit.to_csv("submit_tree.csv", index = False)


In [ ]:
from matplotlib import pyplot as plt

imp = pd.DataFrame(data = tree.feature_importances_,
                   index = X_train.columns,
                   columns = ["important"])
imp.plot.bar(alpha=0.5)
plt.show()

In [ ]:
validation(rfc)
submit = cross_val_and_testing(df=test0, df_test=test, model=rfc, threshold=threshold)
submit.to_csv("submit_rfc.csv", index = False)


In [ ]:
validation(bc)
submit = cross_val_and_testing(df=test0, df_test=test, model=bc, threshold=threshold)
submit.to_csv("submit_bc.csv", index = False)


In [ ]:
validation(abc)
submit = cross_val_and_testing(df=test0, df_test=test, model=abc, threshold=threshold)
submit.to_csv("submit_abc.csv", index = False)


In [ ]:
validation(vc)
submit = cross_val_and_testing(df=test0, df_test=test, model=vc, threshold=threshold)
submit.to_csv("submit_vc.csv", index = False)


In [ ]:
bc1 = BaggingClassifier(base_estimator = bc,  
                        n_estimators = 10,  
                        max_samples = 1.0,  
                        max_features = 1.0,  
                        bootstrap = True,  
                        bootstrap_features = False,  
                        oob_score = False,  
                        warm_start = False,  
                        n_jobs = 1,  
                        random_state = None,  
                        verbose = 0)

validation(bc1)
submit = cross_val_and_testing(df=test0, df_test=test, model=bc1, threshold=threshold)
submit.to_csv("submit_bc1.csv", index = False)


In [ ]:
bc2 = BaggingClassifier(base_estimator = bc1,  
                        n_estimators = 10,  
                        max_samples = 1.0,  
                        max_features = 1.0,  
                        bootstrap = True,  
                        bootstrap_features = False,  
                        oob_score = False,  
                        warm_start = False,  
                        n_jobs = 1,  
                        random_state = None,  
                        verbose = 0)

validation(bc2)
submit = cross_val_and_testing(df=test0, df_test=test, model=bc2, threshold=threshold)
submit.to_csv("submit_bc2.csv", index = False)


1. 描述現象
2. 歸類users
3. 分user建模